In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn

In [3]:
from preprocessing_utils import process_sat_data
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm_notebook as tqdm

In [4]:
data_train = pd.read_csv('../data/train.csv')
data_test = pd.read_csv('../data/track1/test.csv')

In [5]:
train_data = data_train.copy(deep=True)
test_data = data_test.copy(deep=True)

In [6]:
process_sat_data(train_data, test_data, scale=10000)

In [7]:
data_sim = pd.read_csv('../data/data_sim.csv')
data_real = pd.read_csv('../data/data_real.csv')
data_pred = pd.read_csv('../data/data_pred.csv')

# Transformation Matrix Reconstruction

In [8]:
sat_ids = data_sim['0']
ellipse_ids = data_sim['1']

### Real

In [9]:
transformations_real = {}
for sat_id in range(600):
    transformations_real[sat_id] = {}

In [10]:
data_transform_real = data_pred.loc[:, ['r1', 'r2', 'r3', 't1', 't2', 't3']].values

In [11]:
for index, m in enumerate(data_transform_real):
    matrix = R.from_rotvec(m[:3]).as_matrix()
    matrix = np.concatenate([matrix, [m[3:]]], axis=0)
    matrix = np.concatenate([matrix, [[0],[0],[0],[1]]], axis=1)
    transformations_real[sat_ids[index]][ellipse_ids[index]] = matrix

### Sim

In [12]:
transformations_sim = {}
for sat_id in range(600):
    transformations_sim[sat_id] = {}

In [13]:
data_transform_sim = data_sim.loc[:, '5':].values

In [14]:
for index, m in enumerate(data_transform_sim):
    matrix = m.reshape(4,3)
    matrix = np.concatenate([matrix, [[0],[0],[0],[1]]], axis=1)
    transformations_sim[sat_ids[index]][ellipse_ids[index]] = matrix

# Ellipse ID Generation

In [15]:
def get_sat_ellipse_ids(n):
    ids = []
    incomplete = (n%24)
    for i in range(n-incomplete):
        ids.append(int(i/24))
    prev = ids[-1]
    for i in range(incomplete):
        ids.append(prev)
    return ids

In [16]:
def generate_ellipse_ids(data):
    ids = []
    counts = np.unique(data['sat_id'], return_counts=True)[1]
    for count in counts:
        sat_ellipse_ids = get_sat_ellipse_ids(count)
        ids += sat_ellipse_ids
    return ids

In [17]:
cols = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim', 'id', 'sat_id']
data = train_data.loc[:, cols].append(test_data.loc[:, cols])
data['set'] = ['train'] * len(train_data) + ['test']* len(test_data)
data.sort_values('id', inplace=True)
data.reset_index(drop=True, inplace=True)
data['ellipse_id'] = generate_ellipse_ids(data)

# Transformation

In [18]:
def sim_to_real(sat_id, ellipse_id, sim):
    m_sim = transformations_sim[sat_id][ellipse_id]
    m_real = transformations_real[sat_id][ellipse_id]
    pos_sim = np.concatenate([sim[:, :3], np.ones((len(sim), 1))], axis=1)
    vel_sim = np.concatenate([sim[:, 3:], np.zeros((len(sim), 1))], axis=1)
    pos_real = pos_sim.dot(np.linalg.inv(m_sim)).dot(m_real)
    vel_real = vel_sim.dot(np.linalg.inv(m_sim)).dot(m_real)
    real = np.concatenate([pos_sim[:, :3], vel_sim[:, :3]], axis=1).ravel()
    return real

In [19]:
def get_predictions():
    pred = np.array([])
    for sat_id in tqdm(range(600)):
        sat_data = data[data['sat_id'] == sat_id]
        for ellipse_id in range(max(sat_data['ellipse_id'])+1):
            ellipse_data = sat_data[sat_data['ellipse_id'] == ellipse_id].iloc[:, :-4].values
            real_data = sim_to_real(sat_id, ellipse_id, ellipse_data)
            pred = np.append(pred, real_data, axis=0)
    pred = pred.reshape(-1,6)
    pred = pd.DataFrame(pred, columns=['x', 'y', 'z', 'Vx', 'Vy', 'Vz'])
    pred['id'] = data['id']
    pred['sat_id'] = data['sat_id']
    pred['ellipse_id'] = data['ellipse_id']
    pred['set'] = data['set']
    pred.to_csv('../data/pred_real.csv', index=False)
    return pred

In [20]:
compute_real = True
if (compute_real):
    pred_real = get_predictions()
else:
    pred_real = pd.read_csv('../data/pred_real.csv')

# Prediction

In [21]:
pred_real_test = pred_real[pred_real['set'] == 'test']

In [22]:
submission = pred_real_test.loc[:, ['id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']].reset_index(drop=True)

In [23]:
for axis in ['x', 'y', 'z']:
    submission[axis] = submission[axis] * 10000

In [24]:
submission.head(5)

,id,x,y,z,Vx,Vy,Vz
0,3927,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
1,3928,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2,3929,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3,3930,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
4,3931,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724


In [25]:
submission.to_csv('../data/submission.csv', index=False)